In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import mygene
import re

from scipy import stats

In [2]:
# read dataset

df_breast = pd.read_csv("breast.csv", index_col=0)
df_breast.head()

,sample,TCGA-C8-A1HL-01,TCGA-EW-A2FS-01,TCGA-B6-A402-01,TCGA-A2-A3XX-01,GTEX-ZA64-1526-SM-5CVMD,TCGA-Z7-A8R5-01,TCGA-D8-A1JU-01,TCGA-B6-A0RL-01,GTEX-X4EP-2926-SM-3P5YQ,...,TCGA-BH-A18G-01,TCGA-B6-A0IA-01,GTEX-11P7K-0726-SM-5EGKX,TCGA-A8-A090-01,TCGA-5T-A9QA-01,TCGA-AN-A0XT-01,TCGA-A2-A0YK-01,TCGA-A8-A09K-01,TCGA-A2-A1FV-01,GTEX-1117F-2826-SM-5GZXL
0,ENSG00000242268.2,-9.9658,-9.9658,-9.9658,-4.6082,-1.6394,-9.9658,-9.9658,-9.9658,-3.1714,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-3.4580,-9.9658,-3.3076,-3.6259
1,ENSG00000259041.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
2,ENSG00000270112.3,-9.9658,-5.0116,-9.9658,-4.6082,-3.4580,-6.5064,-9.9658,-6.5064,-6.5064,...,-5.5735,-5.0116,-6.5064,-9.9658,-9.9658,-6.5064,-5.0116,-9.9658,-5.5735,-9.9658
3,ENSG00000167578.16,4.9782,5.7035,4.1252,4.8734,4.8679,5.4973,5.4456,5.0343,5.1211,...,4.3793,6.5501,5.0444,5.5503,5.5574,5.0505,4.9626,5.5364,3.9728,4.9402
4,ENSG00000278814.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [3]:
df_breast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60498 entries, 0 to 60497
Columns: 1272 entries, sample to GTEX-1117F-2826-SM-5GZXL
dtypes: float64(1271), object(1)
memory usage: 587.6+ MB


In [4]:
# set index to "sample" column

df_breast = df_breast.set_index("sample")

In [6]:
# transpose the dataset

df = df_breast.T.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1271 entries, TCGA-C8-A1HL-01 to GTEX-1117F-2826-SM-5GZXL
Columns: 60498 entries, ENSG00000242268.2 to ENSG00000181518.3
dtypes: float64(60498)
memory usage: 586.7+ MB


In [38]:
# create a new column "label", cancer=1, healthy=0

df["label"] = df.index.astype(str).str.contains('TCGA-').astype(int)
df.head()

sample,label,ENSG00000242268.2,ENSG00000259041.1,ENSG00000270112.3,ENSG00000167578.16,ENSG00000278814.1,ENSG00000078237.5,ENSG00000269416.5,ENSG00000263642.1,ENSG00000146083.11,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.18,ENSG00000231119.2,ENSG00000280861.1,ENSG00000181518.3
TCGA-C8-A1HL-01,1,-9.9658,-9.9658,-9.9658,4.9782,-9.9658,2.6624,2.0535,-9.9658,3.9580,...,-6.5064,-9.9658,-9.9658,1.0293,-9.9658,-9.9658,4.4095,-5.0116,-9.9658,-9.9658
TCGA-EW-A2FS-01,1,-9.9658,-9.9658,-5.0116,5.7035,-9.9658,1.2696,0.4447,-9.9658,4.2189,...,-3.0469,-9.9658,-9.9658,1.4600,-2.1779,-0.9686,4.4081,-1.8836,-9.9658,-9.9658
TCGA-B6-A402-01,1,-9.9658,-9.9658,-9.9658,4.1252,-9.9658,1.6234,-1.7322,-9.9658,5.0180,...,-5.0116,-9.9658,-9.9658,2.5061,-3.6259,-2.5479,4.4490,-3.4580,-9.9658,-9.9658
TCGA-A2-A3XX-01,1,-4.6082,-9.9658,-4.6082,4.8734,-9.9658,1.5998,-5.5735,-9.9658,4.1859,...,-5.5735,-9.9658,-9.9658,2.0707,-3.8160,-2.6349,4.1078,-1.1811,-9.9658,-9.9658
GTEX-ZA64-1526-SM-5CVMD,0,-1.6394,-9.9658,-3.4580,4.8679,-9.9658,1.9452,-9.9658,-9.9658,4.1804,...,1.8643,-1.9942,-9.9658,1.8484,-2.6349,-9.9658,4.5638,-2.9324,-9.9658,-9.9658


In [85]:
ensembls = df.columns[1:].str.replace(r'\.\d+$', '').tolist()

In [86]:
mg = mygene.MyGeneInfo()
df_mg = mg.getgenes(ensembls, fields="symbol", as_dataframe=True) # fields="symbol,name,summary"

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

In [89]:
# number of genes that has not been found
df_mg['notfound'].notnull().sum()

3745

In [133]:
df_mg.head()

,_id,score,notfound,symbol
query,,,,
ENSG00000242268,ENSG00000242268,20.894310,NaN,LINC02082
ENSG00000259041,ENSG00000259041,20.961487,NaN,AC015722.1
ENSG00000270112,ENSG00000270112,20.978203,NaN,AC090241.2
ENSG00000167578,53916,19.592716,NaN,RAB4B
ENSG00000278814,NaN,NaN,True,NaN


In [155]:
ensembl_symbol_dict = {}

for index, row in df_mg.iterrows():
    ensembl_symbol_dict[index] = row['symbol']

In [191]:
df_genes = df.copy()
df_genes.columns = ['label'] + df_genes.columns[1:].map(lambda val: str(ensembl_symbol_dict[val.split('.')[0]])).tolist()
df_genes.columns

Index(['label', 'LINC02082', 'AC015722.1', 'AC090241.2', 'RAB4B', 'nan',
       'TIGAR', 'LINC01224', 'MIR4802', 'RNF44',
       ...
       'TENM1', 'GABARAPL3', 'AL590824.1', 'BATF3', 'AC010525.1', 'AC097724.1',
       'PPP6R1', 'AL031666.1', 'nan', 'OR8D4'],
      dtype='object', length=60499)

In [197]:
idx = 5
print(df.columns[idx], df_genes.columns[idx])

ENSG00000278814.1 nan


In [201]:
df_genes.head()

,label,LINC02082,AC015722.1,AC090241.2,RAB4B,nan,TIGAR,LINC01224,MIR4802,RNF44,...,TENM1,GABARAPL3,AL590824.1,BATF3,AC010525.1,AC097724.1,PPP6R1,AL031666.1,nan,OR8D4
TCGA-C8-A1HL-01,1,-9.9658,-9.9658,-9.9658,4.9782,-9.9658,2.6624,2.0535,-9.9658,3.9580,...,-6.5064,-9.9658,-9.9658,1.0293,-9.9658,-9.9658,4.4095,-5.0116,-9.9658,-9.9658
TCGA-EW-A2FS-01,1,-9.9658,-9.9658,-5.0116,5.7035,-9.9658,1.2696,0.4447,-9.9658,4.2189,...,-3.0469,-9.9658,-9.9658,1.4600,-2.1779,-0.9686,4.4081,-1.8836,-9.9658,-9.9658
TCGA-B6-A402-01,1,-9.9658,-9.9658,-9.9658,4.1252,-9.9658,1.6234,-1.7322,-9.9658,5.0180,...,-5.0116,-9.9658,-9.9658,2.5061,-3.6259,-2.5479,4.4490,-3.4580,-9.9658,-9.9658
TCGA-A2-A3XX-01,1,-4.6082,-9.9658,-4.6082,4.8734,-9.9658,1.5998,-5.5735,-9.9658,4.1859,...,-5.5735,-9.9658,-9.9658,2.0707,-3.8160,-2.6349,4.1078,-1.1811,-9.9658,-9.9658
GTEX-ZA64-1526-SM-5CVMD,0,-1.6394,-9.9658,-3.4580,4.8679,-9.9658,1.9452,-9.9658,-9.9658,4.1804,...,1.8643,-1.9942,-9.9658,1.8484,-2.6349,-9.9658,4.5638,-2.9324,-9.9658,-9.9658


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1271 entries, TCGA-C8-A1HL-01 to GTEX-1117F-2826-SM-5GZXL
Columns: 60499 entries, ENSG00000242268.2 to label
dtypes: float64(60498), int64(1)
memory usage: 586.7+ MB


In [10]:
# put the label column on first col index

cols = df.columns.tolist()
cols = [cols[-1]]+cols[:-1] # or whatever change you need
df = df.reindex(columns=cols)
df.head()

sample,label,ENSG00000242268.2,ENSG00000259041.1,ENSG00000270112.3,ENSG00000167578.16,ENSG00000278814.1,ENSG00000078237.5,ENSG00000269416.5,ENSG00000263642.1,ENSG00000146083.11,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.18,ENSG00000231119.2,ENSG00000280861.1,ENSG00000181518.3
TCGA-C8-A1HL-01,1,-9.9658,-9.9658,-9.9658,4.9782,-9.9658,2.6624,2.0535,-9.9658,3.9580,...,-6.5064,-9.9658,-9.9658,1.0293,-9.9658,-9.9658,4.4095,-5.0116,-9.9658,-9.9658
TCGA-EW-A2FS-01,1,-9.9658,-9.9658,-5.0116,5.7035,-9.9658,1.2696,0.4447,-9.9658,4.2189,...,-3.0469,-9.9658,-9.9658,1.4600,-2.1779,-0.9686,4.4081,-1.8836,-9.9658,-9.9658
TCGA-B6-A402-01,1,-9.9658,-9.9658,-9.9658,4.1252,-9.9658,1.6234,-1.7322,-9.9658,5.0180,...,-5.0116,-9.9658,-9.9658,2.5061,-3.6259,-2.5479,4.4490,-3.4580,-9.9658,-9.9658
TCGA-A2-A3XX-01,1,-4.6082,-9.9658,-4.6082,4.8734,-9.9658,1.5998,-5.5735,-9.9658,4.1859,...,-5.5735,-9.9658,-9.9658,2.0707,-3.8160,-2.6349,4.1078,-1.1811,-9.9658,-9.9658
GTEX-ZA64-1526-SM-5CVMD,0,-1.6394,-9.9658,-3.4580,4.8679,-9.9658,1.9452,-9.9658,-9.9658,4.1804,...,1.8643,-1.9942,-9.9658,1.8484,-2.6349,-9.9658,4.5638,-2.9324,-9.9658,-9.9658


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1271 entries, TCGA-C8-A1HL-01 to GTEX-1117F-2826-SM-5GZXL
Columns: 60499 entries, label to ENSG00000181518.3
dtypes: float64(60498), int64(1)
memory usage: 586.7+ MB


In [12]:
# check missing values

df.isnull().sum().sum()

0

In [15]:
df.describe()

sample,label,ENSG00000242268.2,ENSG00000259041.1,ENSG00000270112.3,ENSG00000167578.16,ENSG00000278814.1,ENSG00000078237.5,ENSG00000269416.5,ENSG00000263642.1,ENSG00000146083.11,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.18,ENSG00000231119.2,ENSG00000280861.1,ENSG00000181518.3
count,1271.000000,1271.000000,1271.000000,1271.000000,1271.000000,1271.0000,1271.000000,1271.000000,1271.0000,1271.000000,...,1271.000000,1271.000000,1271.000000,1271.000000,1271.000000,1271.000000,1271.000000,1271.000000,1271.0000,1271.000000
mean,0.859166,-8.230196,-9.925300,-8.307256,5.047553,-9.9658,2.343132,-3.745185,-9.9658,4.392976,...,-3.974482,-9.523253,-9.888918,1.300982,-5.535052,-6.454150,5.054296,-2.540537,-9.9658,-9.853739
std,0.347987,3.063972,0.549686,2.281018,0.638643,0.0000,0.900305,3.945200,0.0000,0.733129,...,2.609028,1.709346,0.604319,1.098672,3.754530,3.708155,0.562090,1.259430,0.0000,0.732377
min,0.000000,-9.965800,-9.965800,-9.965800,1.647542,-9.9658,-1.181100,-9.965800,-9.9658,0.856800,...,-9.965800,-9.965800,-9.965800,-2.314700,-9.965800,-9.965800,2.467500,-9.965800,-9.9658,-9.965800
25%,1.000000,-9.965800,-9.965800,-9.965800,4.687100,-9.9658,1.791200,-6.506400,-9.9658,3.976000,...,-5.573500,-9.965800,-9.965800,0.556800,-9.965800,-9.965800,4.726600,-3.171400,-9.9658,-9.965800
50%,1.000000,-9.965800,-9.965800,-9.965800,5.080300,-9.9658,2.395300,-3.046900,-9.9658,4.447000,...,-4.035000,-9.965800,-9.965800,1.287500,-3.625900,-9.965800,5.050500,-2.465900,-9.9658,-9.965800
75%,1.000000,-5.011600,-9.965800,-6.506400,5.449100,-9.9658,2.957150,-0.641600,-9.9658,4.877800,...,-2.388400,-9.965800,-9.965800,2.018300,-2.388400,-2.727400,5.403100,-1.780900,-9.9658,-9.965800
max,1.000000,1.828200,-1.117200,-0.150400,7.150100,-9.9658,5.417900,6.061100,-9.9658,6.634800,...,2.782600,-0.512500,-2.634900,5.397200,1.632700,0.085400,7.858300,3.540929,-9.9658,-3.307600


In [14]:
# save as csv file

df.to_csv("breast_normal_primary_ENSG_2.0.csv")

In [54]:
# pattern is bla - result is None
re.search('Pigos', 'GTEX-GTZA64-152')

# match 2 chars GT
re.search('GT', 'GTEX-GTZA64-152')

# match any 2 chars
re.search('[A-Z]{2}', 'GTEX-ZA64-152')

# match any 2 digits
re.search('[0-9]{2}', 'GTEX-ZA64-152')
re.search('[0123456789]{2}', 'GTEX-ZA64-152')

# match any 2 chars + 2 digits + dash
re.search('[A-Z]{2}[0-9]{2}-', 'GT22-ZA64-152')

# match any 2 chars + 2 digits + dash, 2nd chars is always "A"
re.search('[A-Z][AX][0-9]{2}-', 'GT22-ZA64-152')

# match ALL ugly 4 chars and dash if available - slowly evolving from naive
# \w = [a-zA-Z]
# \d = [0-9]
# . - representing any char except whitespace
# \. - escaped "." which means a character "."
re.findall('\w.\d{2}', 'GT22-Z+64-152-ABC-Z!43')

# [^A-Z] - any char but A-Z set
# a? - match zero or one "a"
# .? - match zero or one "any character"
# a+ - match one or more "a"
# .+ - match one or more "any character"
# a* - match zero or more "a"
# .* - match zero or more "any character"
re.findall('\w[^A-Z]\d{2}', 'GT22-Z+64-152-ABC-Z!43')
'GT22-Z+64-152-ABC-Z!43'.replace('Z+64', '')
'GT22-Z+64-152-ABC-Z!43'.replace('-', '')
re.sub(r'-\w[^A-Z]\d{2}', '', 'GT22-Z+64-152-ABC-Z!43')
re.sub(r'-\w[^A-Z]\d{2}x?', '', 'GT22-Z+64x-152-ABC-Z!43-124-U)43-A@99')
re.sub(r'-\w[^A-Z]\d{2}x+', '', 'GT22-Z+64x-152-ABC-Z!43-124-U)43xx-A@99')
re.sub(r'-\w[^A-Z]\d{2}x{0,2}', '', 'GT22-Z+64x-152-ABC-Z!43-124-U)43xx-A@99')
re.sub(r'-\w[^A-Z]\d{2}x*', '', 'GT22-Z+64x-152-ABC-Z!43-124-U)43xx-A@99')

# ^ - pokud neni v ramci [], tak znamena zacatek radku (ne stringu!! ale radku)
# $ - znamena konec radku
re.search('^TCGA', 'TCGA-A2-A3XX-01')
re.search('^TCGA-.{2}-.{4}$', 'TCGA-A2-A3XX-01') # no match because $ is representing the end of a line and there is -01 remaining

# remove all .XX at the end
re.sub('\..*', '', 'EWGW00001019200124.25')
re.sub('\.\d+', '', 'EWGW00001019200124.435')
re.sub('\.\d?', '', 'EWGW00001019200124.3')
re.sub('(1\d){2,}', '', 'EWGW00101019200124.3')

'EWGW00200124.3'

In [55]:
re.sub('\.\d+', '', 'EWGW00001019200124.43')

'EWGW00001019200124'

In [ ]:
# remove .12 from ensembl number

df["ensembl"].apply(lambda value: re.sub("\.\d+", "", value))